In [43]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [44]:
df= pd.read_csv(r"C:\Users\16479\Downloads\train.csv")

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [46]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
# Handling missing values: fill missing 'age' values with median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Removing duplicates: check for duplicates and drop if any
df.drop_duplicates(inplace=True)

# Correcting data entry errors: replace any negative 'fare' values with the median fare
df.loc[df['Fare'] < 0, 'Fare'] = df['Fare'].median()

# Converting data types: convert 'sex' variable to a categorical data type
df['Sex'] = df['Sex'].astype('category')

# Addressing outliers: remove rows where 'fare' is greater than 3 standard deviations from the mean
fare_mean = df['Fare'].mean()
fare_std = df['Fare'].std()
df = df[(df['Fare'] >= fare_mean - 3 * fare_std) & (df['Fare'] <= fare_mean + 3 * fare_std)]

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 871 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  871 non-null    int64   
 1   Survived     871 non-null    int64   
 2   Pclass       871 non-null    int64   
 3   Name         871 non-null    object  
 4   Sex          871 non-null    category
 5   Age          871 non-null    float64 
 6   SibSp        871 non-null    int64   
 7   Parch        871 non-null    int64   
 8   Ticket       871 non-null    object  
 9   Fare         871 non-null    float64 
 10  Cabin        187 non-null    object  
 11  Embarked     869 non-null    object  
dtypes: category(1), float64(2), int64(5), object(4)
memory usage: 82.6+ KB


In [49]:
df.dropna(subset=['Embarked'], inplace=True)

In [51]:
sex_mapping = {'male': 1, 'female': 0}
df['Sex'] = df['Sex'].map(sex_mapping)

In [52]:
df['Sex'] = df['Sex'].astype(int)

In [53]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [62]:
unique_embarked_values = df['Embarked'].unique()
print(unique_embarked_values)

['S' 'C' 'Q']


In [63]:
embarked_mapping = {'S': 1, 'C': 2, 'Q': 3}

# Replace values using the mapping dictionary
df['Embarked'] = df['Embarked'].replace(embarked_mapping)

# Convert the column to integer data type
df['Embarked'] = df['Embarked'].astype(int)

In [64]:
x = df[['Pclass', 'Sex','Age', 'SibSp', 'Parch','Fare', 'Embarked']]
y = df['Survived']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [66]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

In [72]:
clf = AdaBoostClassifier(n_estimators=75, learning_rate=1.3)
scores = cross_val_score(clf, x, y, cv=5)
scores.mean()

0.814749850508272

In [73]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', AdaBoostClassifier())
])

# Number of bootstrap samples and trees in the ensemble
n_samples = len(X_train)
n_trees = 50

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), size=len(X), replace=True)
    return X[indices], y[indices]

In [74]:
trees = []
for _ in range(n_trees):
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    
    # Clone the pipeline for each tree
    cloned_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
    cloned_pipeline.fit(X_sample, y_sample)
    trees.append(cloned_pipeline)


KeyError: "None of [Index([568, 402, 361,  18,  60, 162,  99,  37, 470, 510,\n       ...\n       253, 242, 186, 514, 277, 244, 347, 214, 378, 215],\n      dtype='int32', length=608)] are in the [columns]"